A short exploration of some metaprogramming techniques for Python, inspired by [Full Monty: Intro to Python Metaprogramming](http://www.oscon.com/oscon2014/public/schedule/detail/34107).

## Decorators

If you've been writing Python for a while, you've almost certainly seen decorators. They look like this:

```python
@decorator
def some_function():
    pass
```

Do you know what that is doing? Here's a hint: it's a shortcut for this formulation:

```python
some_function = decorator(some_function)
```

A decorator is a function that:
 - accepts a *callable* as an argument
 - (hopefully) returns a callable that can (hopefully) be used in place of the original

A *callable* is any object that executes code when you invoke it with parentheses. Functions are callables, but so are classes, methods, types, and any object that has a \_\_call\_\_ method. By "decorating" a callable, you are altering, wrapping, or replacing it.

In [75]:
# a simple function decorator
def silly_decorator(func):
    def better_func(*args, **kwargs):
        if hasattr(func, 'yesiamsure'):
            return func(*args,**kwargs)
        else:
            func.yesiamsure = True
            return "are you sure?"
    return better_func
    

@silly_decorator
def serious_math(a, b):
    return a +b

print serious_math(2,3)
print serious_math(2,3)

are you sure?
5


## the @property decorator

Let us imagine a shopping cart application. Every item in an order is represented by a LineItem, encapsulating the price and quantity.

LineItem offers a "total" method, that returns the total price (quantity * item price).

In [67]:
class LineItem(object):
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity
        

    
    def total(self):
        return self.price * self.quantity
    
bananas = LineItem(1.5, 3)
print bananas.total()

4.5


Yum!

There's a big problem, though:


In [59]:
mangoes = LineItem( 2.0, -3)

print mangoes.total()

-6.0


We shouldn't allow LineItems to be created with negative values.

One way to solve this would be to use the 'property' decorator, which allows you define a "setter" method that is invoked every time you try to set the value, even in \_init\_! 

In [60]:
class LineItem(object):
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity
        

    @property
    def quantity(self):
        return self._quantity
    
    @quantity.setter
    def quantity(self, value):
        if value > 0:
            self._quantity = value
        else:
            raise ValueError("quantity must be greater that 0!")
        
    
    def total(self):
        return self.price * self.quantity
    
mangoes = LineItem(2.0, -3)

ValueError: quantity must be greater that 0!

## Encapsulate property behaviors with descriptors

What if we wanted to do the same thing with prices? We could use do some quick copy and paste work, and implement another property:

In [68]:
class LineItem(object):
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity
        

    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, value):
        if value > 0:
            self._price = value
        else:
            raise ValueError("price must be greater that 0!")
        
        
    @property
    def quantity(self):
        return self._quantity
    
    @quantity.setter
    def quantity(self, value):
        if value > 0:
            self._quantity = value
        else:
            raise ValueError("quantity must be greater that 0!")
        
    
    def total(self):
        return self.price * self.quantity
    


But that is... unsatisfying, and violates [DRY](http://c2.com/cgi/wiki?DontRepeatYourself). It wouldn't be difficult to extract the validation logic into a function or method, but that doesn't buy us much.

The feature Python provides to deal with this sort of situation is called "[descriptors](https://docs.python.org/2/howto/descriptor.html)". With descriptors, a class can delegate setting, getting, and deleting a particular attribute to another object.

In fact, any *instance* attached to a *class*, can be a descriptor, as long as it conforms to the [descriptor protocol](https://docs.python.org/2/howto/descriptor.html#id4).

A descriptor that encapsulates our validation logic might look like:

In [62]:
class PositiveNumber(object):
    def __init__(self, storage_name):
        self.storage_name = storage_name
        
    def __set__(self, obj, value):
        if value > 0:
            obj.__dict__[self.storage_name] = value
        else:
            raise ValueError("%s must be greater that 0!" % self.storage_name)

In [63]:
class LineItem(object):
    price = PositiveNumber('price')
    quantity = PositiveNumber('quantity')
    
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity
    
    def total(self):
        return self.price * self.quantity

Negative prices and negative quantities now fail as expected

In [64]:
prunes = LineItem(-1,1)

ValueError: price must be greater that 0!

In [66]:
apples = LineItem(1,1)
print apples.total()

1
